In [1]:
import tensorflow as tf
import numpy as np 
from sklearn.datasets import fetch_california_housing
from IPython.display import clear_output, Image, display, HTML

###### Do not modify here ###### 
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = graph_def
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
###### Do not modify  here ######

###### Implement Data Preprocess here ######
housing = fetch_california_housing()
# Show description/ statisics about the dataset
print(housing.DESCR)
print('Features')
print('----------')
print(', '.join(housing.feature_names))
print("\nShape of dataset:", housing.data.shape)
print("Shape of label:", housing.target.shape)

California housing dataset.

The original database is available from StatLib

    http://lib.stat.cmu.edu/

The data contains 20,640 observations on 9 variables.

This dataset contains the average house value as target variable
and the following input variables (features): average income,
housing average age, average rooms, average bedrooms, population,
average occupation, latitude, and longitude in that order.

References
----------

Pace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,
Statistics and Probability Letters, 33 (1997) 291-297.


Features
----------
MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup, Latitude, Longitude

Shape of dataset: (20640, 8)
Shape of label: (20640,)


## Get training set and testing set

In [2]:
cut = int(housing.data.shape[0]*0.9)
x_train, x_test = housing.data[:cut], housing.data[cut:]
y_train, y_test = housing.target[:cut], housing.target[cut:]

print("\nShape of training set:", x_train.shape)
print("Shape of testing set:", x_test.shape)
print("Shape of training label:", y_train.shape)
print("Shape of testing label:", y_test.shape)


Shape of training set: (18576, 8)
Shape of testing set: (2064, 8)
Shape of training label: (18576,)
Shape of testing label: (2064,)


## Define Model Graph

In [14]:
# Clear graph
tf.reset_default_graph()

# Define variables to take input feature x, label y
x = tf.placeholder(tf.float64, shape = [None, x_train.shape[1]], name="Input")
y = tf.placeholder(tf.float64, name="Y")
# y = tf.placeholder(tf.float32, shape = [None, 1], name="Input")

# Define the weights for each element of input x, and random assign value as normal distribution
W = tf.Variable(tf.random_uniform([x_train.shape[1], 1], -1, 1, tf.float64), name="Weights")
# W = tf.Variable(tf.zeros([8,1]), dtype=tf.float32)

# Define the bias for the linear function
b = tf.Variable([0.0], dtype=tf.float64, name="Bias")

# Connect each elements for model
linear_regression = tf.reshape(tf.matmul(x, W), [-1], name="Convert_dim") + b
# linear_regression = tf.matmul(x, W) + b

# Define cost/loss function
error_rate = tf.abs((y - linear_regression)/y , name="Absolute_value")
# error_rate = (y - linear_regression)/y 
loss = tf.reduce_mean(error_rate, name="Sum_Error")

# Define training optimizer
optimizer = tf.train.GradientDescentOptimizer(0.05)
train = optimizer.minimize(loss)


## Model Graph

In linear regression, we need to model a line following the equation: 

$y = \sum_{i=1}^n x_i w_i + b$ 

Where $i$ stand for each feature, and we given a weight $w$ for each $x_i$ in $x$.

We will go through this graph in bottom up direction:

1. In bottom layer, we got **Input** and **Weights** elements.
  
  a. Input, represent $x$ , is the input node which we input our feature about the houses
  
  b. Weights, represent $w$, is the weight  we given to each feature $x_i$. To initialized the weight value, we using random_uniform method to give a set of value($-1 \leq w_i \leq 1$) follow uniform distribution.

2. Do the matrix mutiple by utilizing tf.matmul function to calculate $\sum_{i=1}^n x_i w_i$

3. ff

In [15]:
###### Start TF session ######
with tf.Session() as sess:
    show_graph(tf.get_default_graph().as_graph_def())

## Directly Train
Let's try directly train model without data pre-processing

In [16]:
###### Start TF session ######
with tf.Session() as sess:
    # training loop
    sess.run(tf.global_variables_initializer()) # reset values to wrong
    for i in range(3000):
        sess.run(train, {x: x_train, y: y_train})
        if i % 600 == 0:
            print("Round ",i+1)
            l = sess.run(loss, {x: x_train, y: y_train})
            print("Training - loss: %f"%(l/ y_train.shape[0]))
            l = sess.run(loss, {x: x_test, y: y_test})
            print("Testing  - loss: %f\n"%(l/ y_test.shape[0]))
            
    # evaluate training accuracy
    y_, e_, l = sess.run([y, error_rate, loss], {x: x_train, y: y_train})
    print("Training - loss: %f"%(l/ y_train.shape[0]))
    lr_1, e_1, l1 = sess.run([linear_regression, error_rate, loss], {x: x_test, y: y_test})
    print("Testing  - loss: %f"%(l1/ y_test.shape[0]))
    
#     show_graph(tf.get_default_graph().as_graph_def())

Round  1
Training - loss: 2.212990
Testing  - loss: 23.614373

Round  601
Training - loss: 2.204763
Testing  - loss: 23.523639

Round  1201
Training - loss: 2.204763
Testing  - loss: 23.523639

Round  1801
Training - loss: 2.204763
Testing  - loss: 23.523639

Round  2401
Training - loss: 2.204763
Testing  - loss: 23.523639

Training - loss: 0.037354
Testing  - loss: 0.397945


## Train after pre-processing
### Pre-processing

In [6]:
import pandas as pd

In [7]:
x_train_pd = pd.DataFrame(x_train, columns=housing.feature_names)
x_train_pd.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,18576.000000,18576.000000,18576.000000,18576.000000,18576.000000,18576.000000,18576.000000,18576.000000
mean,3.917221,29.118594,5.403187,1.095654,1433.185831,3.020782,35.430371,-119.409006
std,1.932617,12.658810,2.543630,0.485395,1142.294876,6.078652,2.061388,1.993468
min,0.499900,1.000000,0.846154,0.375000,3.000000,0.692308,32.540000,-124.350000
25%,2.585725,19.000000,4.391440,1.005917,793.000000,2.421582,33.900000,-121.540000
50%,3.571900,30.000000,5.188982,1.048474,1170.000000,2.819232,34.160000,-118.370000
75%,4.803600,38.000000,6.031433,1.098765,1727.000000,3.296951,37.580000,-117.950000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,599.714286,41.950000,-114.310000


### Standardization

In [8]:
# Training data
x_train_standard_pd = x_train_pd.sub(x_train_pd.mean(),axis=1).div(x_train_pd.std(),axis=1)
x_train_standard = x_train_standard_pd.values
x_train_standard_pd

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,2.280834,0.938588,0.621529,-0.148012,-0.972766,-0.076534,1.188340,-1.415119
1,2.268519,-0.641339,0.328251,-0.254995,0.847254,-0.149859,1.178637,-1.410103
2,1.728319,1.807548,1.134185,-0.045751,-0.820441,-0.035949,1.173786,-1.420135
3,0.893027,1.807548,0.162824,-0.046549,-0.766165,-0.077786,1.173786,-1.425152
4,-0.036749,1.807548,0.345438,-0.030022,-0.760037,-0.138076,1.173786,-1.425152
5,0.061874,1.807548,-0.252210,0.016426,-0.893102,-0.144915,1.173786,-1.425152
6,-0.133560,1.807548,-0.185279,-0.297267,-0.296934,-0.146805,1.168935,-1.425152
7,-0.412508,1.807548,-0.238109,-0.069696,-0.241782,-0.202763,1.168935,-1.425152
8,-0.950432,1.017584,-0.436018,0.045310,-0.198885,-0.163505,1.168935,-1.430168
9,-0.116951,1.807548,-0.170071,-0.217261,0.103138,-0.139589,1.168935,-1.425152


### Value change between standardization before and after

In [9]:
print(x_train[0])
print(x_train_standard[0])

[   8.3252       41.            6.98412698    1.02380952  322.
    2.55555556   37.88       -122.23      ]
[ 2.28083372  0.9385879   0.62152884 -0.14801185 -0.97276619 -0.07653445
  1.18833965 -1.415119  ]


In [10]:
# Testing data
x_test_pd = pd.DataFrame(x_test, columns=housing.feature_names)
x_test_standard = x_test_pd.sub(x_train_pd.mean(),axis=1).div(x_train_pd.std(),axis=1).values


### Training

In [17]:
###### Start TF session ######
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) # re-assign Weights
    # training loop
    for i in range(3000):
        sess.run(train, {x: x_train_standard, y: y_train})
        if i % 600 == 0:
            print("Round ",i+1)
            l = sess.run(loss, {x: x_train_standard, y: y_train})
            print("Training - loss: %f"%(l/ y_train.shape[0]))
            l = sess.run(loss, {x: x_test_standard, y: y_test})
            print("Testing  - loss: %f\n"%(l/ y_test.shape[0]))
            
    # evaluate training accuracy
    lr_, e_, l = sess.run([linear_regression, error_rate, loss], {x: x_train_standard, y: y_train})
    print("Training - loss: %f"%(l/ y_train.shape[0]))
    lr_1, e_1, l1 = sess.run([linear_regression, error_rate, loss], {x: x_test_standard, y: y_test})
    print("Testing  - loss: %f"%(l1/ y_test.shape[0]))

Round  1
Training - loss: 0.000063
Testing  - loss: 0.000624

Round  601
Training - loss: 0.000014
Testing  - loss: 0.000131

Round  1201
Training - loss: 0.000014
Testing  - loss: 0.000136

Round  1801
Training - loss: 0.000014
Testing  - loss: 0.000138

Round  2401
Training - loss: 0.000014
Testing  - loss: 0.000139

Training - loss: 0.000014
Testing  - loss: 0.000139
